In [3]:
import glob
import subprocess
import shlex
from collections import namedtuple

In [4]:
UNKNOWN = -1
class FrameMeta:
    def __init__(self, frame_no=-1, delay=-1, height=-1, width=-1):
        self.frame_no = frame_no
        self.delay = delay
        self.height = height
        self.width = width
    
    def __repr__(self):
        return f"#{self.frame_no} -d {self.delay}s hxw {self.height}x{self.width}"



def run_linux_cmd(cmd):
    # print (cmd)
    full_cmd = "bash.exe -c '"+ cmd +"'"
    process = subprocess.Popen(shlex.split(full_cmd), stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    out, err = process.communicate()
    out = out.decode("utf-8")
    err = err.decode("utf-8")
    return out, err

def get_frames_meta(filename):
    out, err = run_linux_cmd(f"gifsicle -I \"{filename}\"")
    meta = []
    imgline = False
    for line in out.splitlines():
        line = line.strip()
        if line.startswith("+"):
            tokens = line.split(' ')
            m = FrameMeta()
            m.frame_no = int(tokens[2][1:])
            hxw = tokens[3].split("x")
            m.height=int(hxw[0])
            m.width=int(hxw[1])
            meta.append(m)
            imgline = True
        else:
            if imgline:
                tokens = line.split(' ')
                delay_t = tokens.index("delay")
                meta[-1].delay = float(tokens[delay_t + 1][:-1])
            imgline = False
    return meta

def combine_durations(meta, min_duration):
    new_meta = []
    current_duration = float("+inf")
    for m in meta:
        if current_duration < min_duration:
            current_duration += m.delay
        else:
            if len(new_meta):
                new_meta[-1].delay = current_duration
            new_meta.append(m)
            current_duration = m.delay
    return new_meta

def build_durations_request(new_meta):
    return " ".join(f'"#{m.frame_no}" -d {int(m.delay * 100)}' for m in new_meta)

def get_resize_cmd(meta, target_size=64):
    if len(meta):
        m = meta[0]
        if m.height == -1 or m.width == -1:
            return ""
        if m.height > target_size or m.width > target_size:
            return f"--resize-fit {target_size}x{target_size}"
    return ""

        


In [5]:
for f in glob.glob("./fun_images/giphy_sources/*.gif"):
    f = f.replace("\\", "/")
    new_filename = f.split("/")
    new_filename.remove("giphy_sources")
    new_filename = "/".join(new_filename)
    print(new_filename)
    try:
        meta = get_frames_meta(f)
        # print(meta)
        # break
        new_meta = combine_durations(meta, 1/15)
        durations_request = build_durations_request(new_meta)
        resize_cmd = get_resize_cmd(new_meta)
        cmd = f"gifsicle -U \"{f}\" {durations_request} {resize_cmd} -o \"{new_filename}\""
        run_linux_cmd(cmd)
    except Exception as e:
        print(f"Unable to convert {f}, copying: {e}")
        cmd = f"cp \"{f}\" \"{new_filename}\""
        run_linux_cmd(cmd)
    


./fun_images/1.gif
./fun_images/10.gif
./fun_images/12.gif
./fun_images/13.gif
./fun_images/14.gif
./fun_images/15.gif
Unable to convert ./fun_images/giphy_sources/15.gif, copying: 'delay' is not in list
./fun_images/16.gif
./fun_images/17.gif
./fun_images/2.gif
./fun_images/3.gif
./fun_images/4.gif
./fun_images/5.gif
./fun_images/6.gif
./fun_images/7.gif
./fun_images/8.gif
./fun_images/9.gif
./fun_images/central.gif
./fun_images/hexagons.gif
./fun_images/mondrian.gif
./fun_images/spiral.gif
./fun_images/spiral2.gif
./fun_images/squares.gif
./fun_images/squares2.gif


In [6]:
target_dir = "${HOME}/win_home/YandexDisk/backpack/"
cmd = f"cp ./fun_images/*.gif {target_dir}"
run_linux_cmd(cmd)
out, err = run_linux_cmd(f"ls -lh {target_dir}")
print(out)

total 3.3M
-rwxrwxrwx 1 dmitrym dmitrym  40K Apr  6 14:05 1.gif
-rwxrwxrwx 1 dmitrym dmitrym  22K Apr  6 14:05 10.gif
-rwxrwxrwx 1 dmitrym dmitrym  31K Apr  6 14:05 12.gif
-rwxrwxrwx 1 dmitrym dmitrym  23K Apr  6 14:05 13.gif
-rwxrwxrwx 1 dmitrym dmitrym  28K Apr  6 14:05 14.gif
-rwxrwxrwx 1 dmitrym dmitrym 1.9M Apr  6 14:05 15.gif
-rwxrwxrwx 1 dmitrym dmitrym  27K Apr  6 14:05 16.gif
-rwxrwxrwx 1 dmitrym dmitrym  51K Apr  6 14:05 17.gif
-rwxrwxrwx 1 dmitrym dmitrym  41K Apr  6 14:05 2.gif
-rwxrwxrwx 1 dmitrym dmitrym  28K Apr  6 14:05 3.gif
-rwxrwxrwx 1 dmitrym dmitrym  18K Apr  6 14:05 4.gif
-rwxrwxrwx 1 dmitrym dmitrym 202K Apr  6 14:05 5.gif
-rwxrwxrwx 1 dmitrym dmitrym  53K Apr  6 14:05 6.gif
-rwxrwxrwx 1 dmitrym dmitrym 2.6K Apr  6 14:05 7.gif
-rwxrwxrwx 1 dmitrym dmitrym 7.2K Apr  6 14:05 8.gif
-rwxrwxrwx 1 dmitrym dmitrym  52K Apr  6 14:05 9.gif
-rwxrwxrwx 1 dmitrym dmitrym 101K Apr  6 14:05 central.gif
-rwxrwxrwx 1 dmitrym dmitrym  59K Apr  6 14:05 hexagons.gif
-rwxrwxrwx 1 dm

In [66]:
line = "adv  "

In [99]:
a = [1, 2 ,3]

In [85]:
a.index(1)

0

In [100]:
a.`

Signature: a.remove(value, /)
Docstring:
Remove first occurrence of value.

Raises ValueError if the value is not present.
Type:      builtin_function_or_method


In [107]:
f.split("/").remove("giphy_sources")